In [47]:
import re
import itertools
import numpy as np
import pandas as pd
import torch
from transformers import (
#    AutoModel, 
    AutoTokenizer, 
#    BertTokenizer,
)

torch.set_grad_enabled(False)

In [48]:
# Store the model we want to use
MODEL_NAME = "bert-base-uncased" #@param

# We need to create the model and tokenizer
#model = AutoModel.from_pretrained(MODEL_NAME,
#                                  output_hidden_states=True,
#                                  output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [52]:
sentences = [
    'Do not compare apples and oranges, but apples and apples.',
    'This is a follow-up sentence with less common punctuation!',
    'Does any of this really work?',
    "Do not and don't are subtly different.",
    '"Go away!" — she said.'
]

for sent in sentences:
    print("SENTENCE", sent)
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    inputs = encoded.input_ids
#    print("INPUTS", inputs.shape, inputs)
    decoded = tokenizer.convert_ids_to_tokens(inputs[0], skip_special_tokens=False)
#    print("DECODE", decoded)
    attention_mask =  encoded['attention_mask']
#    print("MASK", attention_mask.shape, attention_mask)
    output = model(inputs, attention_mask)
    token_len = attention_mask.sum().item()
#    print("# OF TOKENS", token_len)
    word_indices = np.array(list(map(lambda e: -1 if e is None else e, encoded.word_ids())))[1:token_len - 1]
#    print("WORD INDICES", word_indices)
    word_groups = np.split(np.arange(word_indices.shape[0]) + 1, np.unique(word_indices, return_index=True)[1])[1:]
#    print("GROUPS", len(word_groups), word_groups)
    
    display(pd.DataFrame([
        (
                g,
                np.array(decoded)[g],
                inputs[0, :][g].numpy().tolist(),
                t,
                "".join(list(map(lambda t: t[2:] if t[:2] == "##" else t, np.array(decoded)[g])))
        )
            for g, t in itertools.zip_longest(word_groups, re.findall(r"\w+|[^\s\w]", sent))
    ], columns=['group', 'token_ids', 'decoded', 'regex', 'combined'])
           )
    print("=" * 40)


SENTENCE Do not compare apples and oranges, but apples and apples.


,group,token_ids,decoded,regex,combined
0,[1],[do],[2079],Do,do
1,[2],[not],[2025],not,not
2,[3],[compare],[12826],compare,compare
3,[4],[apples],[18108],apples,apples
4,[5],[and],[1998],and,and
5,"[6, 7]","[orange, ##s]","[4589, 2015]",oranges,oranges
6,[8],"[,]",[1010],",",","
7,[9],[but],[2021],but,but
8,[10],[apples],[18108],apples,apples
9,[11],[and],[1998],and,and


SENTENCE This is a follow-up sentence with less common punctuation!


,group,token_ids,decoded,regex,combined
0,[1],[this],[2023],This,this
1,[2],[is],[2003],is,is
2,[3],[a],[1037],a,a
3,[4],[follow],[3582],follow,follow
4,[5],[-],[1011],-,-
5,[6],[up],[2039],up,up
6,[7],[sentence],[6251],sentence,sentence
7,[8],[with],[2007],with,with
8,[9],[less],[2625],less,less
9,[10],[common],[2691],common,common


SENTENCE Does any of this really work?


,group,token_ids,decoded,regex,combined
0,[1],[does],[2515],Does,does
1,[2],[any],[2151],any,any
2,[3],[of],[1997],of,of
3,[4],[this],[2023],this,this
4,[5],[really],[2428],really,really
5,[6],[work],[2147],work,work
6,[7],[?],[1029],?,?


SENTENCE Do not and don't are subtly different.


,group,token_ids,decoded,regex,combined
0,[1],[do],[2079],Do,do
1,[2],[not],[2025],not,not
2,[3],[and],[1998],and,and
3,[4],[don],[2123],don,don
4,[5],['],[1005],','
5,[6],[t],[1056],t,t
6,[7],[are],[2024],are,are
7,[8],[subtly],[28797],subtly,subtly
8,[9],[different],[2367],different,different
9,[10],[.],[1012],.,.


SENTENCE "Go away!" — she said.


,group,token_ids,decoded,regex,combined
0,[1],"[""]",[1000],"""",""""
1,[2],[go],[2175],Go,go
2,[3],[away],[2185],away,away
3,[4],[!],[999],!,!
4,[5],"[""]",[1000],"""",""""
5,[6],[—],[1517],—,—
6,[7],[she],[2016],she,she
7,[8],[said],[2056],said,said
8,[9],[.],[1012],.,.
